<a href="https://colab.research.google.com/github/jeremiahoclark/python-coding-patterns/blob/main/05_data_processing_analysis_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing and Analysis Patterns

This notebook covers essential patterns for data processing and analysis in Python, including ETL pipelines, vectorization techniques, and memory-efficient streaming patterns.

## 1. ETL Pipeline Pattern

The ETL (Extract, Transform, Load) pipeline pattern provides a structured approach to data processing workflows.

In [ ]:
from typing import Iterator, List, Dict, Any, Callable
import pandas as pd
import json
from datetime import datetime
import logging

class ETLPipeline:
    def __init__(self):
        self.extractors = []
        self.transformers = []
        self.loaders = []
        self.logger = logging.getLogger(__name__)

    def add_extractor(self, extractor: Callable):
        self.extractors.append(extractor)
        return self

    def add_transformer(self, transformer: Callable):
        self.transformers.append(transformer)
        return self

    def add_loader(self, loader: Callable):
        self.loaders.append(loader)
        return self

    def execute(self):
        data = None

        # Extract phase
        for extractor in self.extractors:
            try:
                extracted_data = extractor()
                data = extracted_data if data is None else self._merge_data(data, extracted_data)
                self.logger.info(f"Extraction completed: {extractor.__name__}")
            except Exception as e:
                self.logger.error(f"Extraction failed: {extractor.__name__} - {e}")
                raise

        # Transform phase
        for transformer in self.transformers:
            try:
                data = transformer(data)
                self.logger.info(f"Transformation completed: {transformer.__name__}")
            except Exception as e:
                self.logger.error(f"Transformation failed: {transformer.__name__} - {e}")
                raise

        # Load phase
        for loader in self.loaders:
            try:
                loader(data)
                self.logger.info(f"Loading completed: {loader.__name__}")
            except Exception as e:
                self.logger.error(f"Loading failed: {loader.__name__} - {e}")
                raise

        return data

    def _merge_data(self, data1, data2):
        if isinstance(data1, pd.DataFrame) and isinstance(data2, pd.DataFrame):
            return pd.concat([data1, data2], ignore_index=True)
        elif isinstance(data1, list) and isinstance(data2, list):
            return data1 + data2
        else:
            return [data1, data2]

In [ ]:
# Example usage of ETL Pipeline
import random
from datetime import datetime, timedelta

# Sample data extractors
def extract_sales_data():
    """Extract sales data from source"""
    return pd.DataFrame({
        'date': [datetime.now() - timedelta(days=i) for i in range(10)],
        'product': [f'Product_{i%3}' for i in range(10)],
        'sales': [random.randint(100, 1000) for _ in range(10)],
        'region': [random.choice(['North', 'South', 'East', 'West']) for _ in range(10)]
    })

def extract_inventory_data():
    """Extract inventory data from source"""
    return pd.DataFrame({
        'product': ['Product_0', 'Product_1', 'Product_2'],
        'stock': [500, 300, 750],
        'cost': [10.5, 15.2, 8.9]
    })

# Sample transformers
def clean_and_validate(data):
    """Clean and validate the data"""
    if isinstance(data, list) and len(data) == 2:
        sales_df, inventory_df = data
        # Merge sales and inventory data
        merged = sales_df.merge(inventory_df, on='product', how='left')
        # Remove any rows with missing data
        cleaned = merged.dropna()
        return cleaned
    return data

def calculate_metrics(data):
    """Calculate business metrics"""
    data = data.copy()
    data['revenue'] = data['sales'] * data['cost']
    data['profit_margin'] = (data['revenue'] - (data['sales'] * data['cost'] * 0.7)) / data['revenue']
    return data

# Sample loaders
def save_to_csv(data):
    """Save data to CSV file"""
    data.to_csv('/tmp/processed_sales_data.csv', index=False)
    print(f"Data saved to CSV: {len(data)} rows")

def print_summary(data):
    """Print data summary"""
    print("\nData Summary:")
    print(f"Total rows: {len(data)}")
    print(f"Total revenue: ${data['revenue'].sum():.2f}")
    print(f"Average profit margin: {data['profit_margin'].mean():.2%}")

# Create and execute pipeline
pipeline = (ETLPipeline()
           .add_extractor(extract_sales_data)
           .add_extractor(extract_inventory_data)
           .add_transformer(clean_and_validate)
           .add_transformer(calculate_metrics)
           .add_loader(save_to_csv)
           .add_loader(print_summary))

# Execute the pipeline
result = pipeline.execute()
print("\nFirst 5 rows of processed data:")
print(result.head())

## 2. Vectorization and Batch Processing

Vectorization patterns leverage NumPy and pandas for efficient batch operations on large datasets.

In [ ]:
import numpy as np
import pandas as pd
import time
from typing import Union, Callable

class VectorizedProcessor:
    """Demonstrates vectorized operations for efficient data processing"""

    @staticmethod
    def scalar_operation(data: list, operation: Callable) -> list:
        """Slow scalar operation for comparison"""
        return [operation(x) for x in data]

    @staticmethod
    def vectorized_operation(data: np.ndarray, operation: Callable) -> np.ndarray:
        """Fast vectorized operation"""
        return operation(data)

    @staticmethod
    def batch_process(data: np.ndarray, batch_size: int, operation: Callable) -> np.ndarray:
        """Process data in batches to manage memory"""
        results = []
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            batch_result = operation(batch)
            results.append(batch_result)
        return np.concatenate(results)

# Performance comparison example
def complex_calculation(x):
    """A complex mathematical operation"""
    return np.sqrt(x**2 + np.sin(x) * np.cos(x))

# Generate test data
size = 1000000
test_data_list = list(range(size))
test_data_array = np.array(test_data_list, dtype=np.float64)

print(f"Processing {size:,} numbers...\n")

# Scalar operation (slow)
start_time = time.time()
scalar_result = VectorizedProcessor.scalar_operation(test_data_list[:10000], complex_calculation)  # Only 10k for demo
scalar_time = time.time() - start_time
print(f"Scalar operation (10k items): {scalar_time:.4f} seconds")

# Vectorized operation (fast)
start_time = time.time()
vectorized_result = VectorizedProcessor.vectorized_operation(test_data_array, complex_calculation)
vectorized_time = time.time() - start_time
print(f"Vectorized operation (1M items): {vectorized_time:.4f} seconds")

# Batch processing
start_time = time.time()
batch_result = VectorizedProcessor.batch_process(test_data_array, 50000, complex_calculation)
batch_time = time.time() - start_time
print(f"Batch processing (1M items, 50k batch): {batch_time:.4f} seconds")

print(f"\nSpeedup factor: {(scalar_time * 100):.1f}x faster with vectorization")

In [ ]:
# Pandas vectorization examples
class PandasVectorization:
    """Demonstrates pandas vectorization techniques"""

    def __init__(self, size: int = 100000):
        # Create sample dataset
        self.df = pd.DataFrame({
            'user_id': np.random.randint(1, 10000, size),
            'age': np.random.randint(18, 80, size),
            'income': np.random.randint(30000, 200000, size),
            'spending': np.random.randint(1000, 50000, size),
            'category': np.random.choice(['A', 'B', 'C', 'D'], size),
            'timestamp': pd.date_range('2023-01-01', periods=size, freq='1min')
        })

    def slow_apply_method(self) -> pd.DataFrame:
        """Slow row-by-row processing"""
        def calculate_score(row):
            base_score = row['spending'] / row['income']
            age_factor = 1.0 if row['age'] < 30 else 0.8 if row['age'] < 50 else 0.6
            category_bonus = {'A': 1.2, 'B': 1.1, 'C': 1.0, 'D': 0.9}[row['category']]
            return base_score * age_factor * category_bonus

        result = self.df.copy()
        result['score'] = result.apply(calculate_score, axis=1)
        return result

    def fast_vectorized_method(self) -> pd.DataFrame:
        """Fast vectorized processing"""
        result = self.df.copy()

        # Vectorized calculations
        base_score = result['spending'] / result['income']

        # Vectorized conditional logic
        age_factor = np.where(result['age'] < 30, 1.0,
                             np.where(result['age'] < 50, 0.8, 0.6))

        # Vectorized mapping
        category_mapping = {'A': 1.2, 'B': 1.1, 'C': 1.0, 'D': 0.9}
        category_bonus = result['category'].map(category_mapping)

        result['score'] = base_score * age_factor * category_bonus
        return result

    def demonstrate_performance(self):
        """Compare performance of different approaches"""
        print(f"Dataset size: {len(self.df):,} rows\n")

        # Test apply method (on smaller subset for demo)
        small_processor = PandasVectorization(10000)
        start_time = time.time()
        apply_result = small_processor.slow_apply_method()
        apply_time = time.time() - start_time
        print(f"Apply method (10k rows): {apply_time:.4f} seconds")

        # Test vectorized method
        start_time = time.time()
        vectorized_result = self.fast_vectorized_method()
        vectorized_time = time.time() - start_time
        print(f"Vectorized method (100k rows): {vectorized_time:.4f} seconds")

        print(f"\nEstimated speedup: ~{(apply_time * 10):.1f}x faster")

        return vectorized_result

# Demonstrate pandas vectorization
processor = PandasVectorization()
result = processor.demonstrate_performance()
print("\nSample results:")
print(result[['age', 'income', 'spending', 'category', 'score']].head())

## 3. Memory Efficiency and Streaming Patterns

These patterns help process large datasets that don't fit in memory by using generators and streaming techniques.

In [ ]:
import csv
import gzip
from typing import Iterator, Generator, Any
import os
import tempfile

class StreamingProcessor:
    """Memory-efficient streaming data processor"""

    @staticmethod
    def read_large_file(file_path: str, chunk_size: int = 8192) -> Generator[str, None, None]:
        """Read large file in chunks to avoid memory issues"""
        with open(file_path, 'r') as file:
            while True:
                chunk = file.read(chunk_size)
                if not chunk:
                    break
                yield chunk

    @staticmethod
    def read_csv_streaming(file_path: str) -> Generator[Dict[str, Any], None, None]:
        """Stream CSV rows one at a time"""
        with open(file_path, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                yield row

    @staticmethod
    def process_streaming_data(data_stream: Iterator,
                             processor_func: Callable,
                             batch_size: int = 1000) -> Generator[Any, None, None]:
        """Process streaming data in batches"""
        batch = []
        for item in data_stream:
            batch.append(item)
            if len(batch) >= batch_size:
                yield processor_func(batch)
                batch = []

        # Process remaining items
        if batch:
            yield processor_func(batch)

    @staticmethod
    def windowed_processing(data_stream: Iterator, window_size: int) -> Generator[List, None, None]:
        """Process data in sliding windows"""
        window = []
        for item in data_stream:
            window.append(item)
            if len(window) > window_size:
                window.pop(0)
            if len(window) == window_size:
                yield window.copy()

# Create sample data for demonstration
def create_sample_csv(file_path: str, num_rows: int = 100000):
    """Create a sample CSV file for streaming demonstration"""
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'value', 'category', 'timestamp'])
        for i in range(num_rows):
            writer.writerow([
                i,
                random.randint(1, 1000),
                random.choice(['A', 'B', 'C']),
                f'2023-01-{(i % 30) + 1:02d}'
            ])

# Demonstration
temp_file = '/tmp/sample_data.csv'
create_sample_csv(temp_file, 50000)
print(f"Created sample file: {temp_file}")
print(f"File size: {os.path.getsize(temp_file) / 1024 / 1024:.2f} MB")

In [ ]:
# Streaming processing examples
def aggregate_batch(batch: List[Dict]) -> Dict:
    """Aggregate a batch of records"""
    if not batch:
        return {}

    total_value = sum(int(row['value']) for row in batch)
    categories = {}
    for row in batch:
        cat = row['category']
        categories[cat] = categories.get(cat, 0) + 1

    return {
        'batch_size': len(batch),
        'total_value': total_value,
        'avg_value': total_value / len(batch),
        'categories': categories
    }

# Process the CSV file in streaming fashion
print("Processing CSV file in streaming mode...")
stream = StreamingProcessor.read_csv_streaming(temp_file)
processed_batches = StreamingProcessor.process_streaming_data(
    stream, aggregate_batch, batch_size=5000
)

# Collect results
batch_count = 0
total_records = 0
grand_total_value = 0

for batch_result in processed_batches:
    batch_count += 1
    total_records += batch_result['batch_size']
    grand_total_value += batch_result['total_value']

    if batch_count <= 3:  # Show first 3 batches
        print(f"\nBatch {batch_count}:")
        print(f"  Records: {batch_result['batch_size']}")
        print(f"  Average value: {batch_result['avg_value']:.2f}")
        print(f"  Categories: {batch_result['categories']}")

print(f"\nSummary:")
print(f"Total batches processed: {batch_count}")
print(f"Total records: {total_records:,}")
print(f"Grand total value: {grand_total_value:,}")
print(f"Overall average: {grand_total_value / total_records:.2f}")

In [ ]:
# Advanced streaming pattern: Real-time data processing
class RealTimeProcessor:
    """Simulates real-time data processing with memory efficiency"""

    def __init__(self, window_size: int = 100):
        self.window_size = window_size
        self.metrics = {
            'total_processed': 0,
            'current_avg': 0,
            'peak_value': 0,
            'anomaly_count': 0
        }

    def process_data_stream(self, data_generator: Generator) -> Generator[Dict, None, None]:
        """Process streaming data with sliding window analytics"""
        window = []

        for data_point in data_generator:
            value = float(data_point['value'])

            # Update sliding window
            window.append(value)
            if len(window) > self.window_size:
                window.pop(0)

            # Update metrics
            self.metrics['total_processed'] += 1
            self.metrics['current_avg'] = sum(window) / len(window)
            self.metrics['peak_value'] = max(self.metrics['peak_value'], value)

            # Detect anomalies (values > 2 std devs from mean)
            if len(window) >= 10:
                window_mean = sum(window) / len(window)
                window_std = (sum((x - window_mean) ** 2 for x in window) / len(window)) ** 0.5
                if abs(value - window_mean) > 2 * window_std:
                    self.metrics['anomaly_count'] += 1

            # Yield results periodically
            if self.metrics['total_processed'] % 1000 == 0:
                yield {
                    'timestamp': data_point.get('timestamp', 'unknown'),
                    'current_value': value,
                    'window_avg': self.metrics['current_avg'],
                    'total_processed': self.metrics['total_processed'],
                    'anomaly_count': self.metrics['anomaly_count'],
                    'peak_value': self.metrics['peak_value']
                }

# Demonstrate real-time processing
print("\nDemonstrating real-time streaming processing...")
processor = RealTimeProcessor(window_size=50)
data_stream = StreamingProcessor.read_csv_streaming(temp_file)

results_count = 0
for result in processor.process_data_stream(data_stream):
    results_count += 1
    if results_count <= 5:  # Show first 5 periodic results
        print(f"\nResult {results_count}:")
        print(f"  Processed: {result['total_processed']:,} records")
        print(f"  Current value: {result['current_value']}")
        print(f"  Window average: {result['window_avg']:.2f}")
        print(f"  Anomalies detected: {result['anomaly_count']}")
        print(f"  Peak value: {result['peak_value']}")

print(f"\nFinal metrics:")
print(f"Total processed: {processor.metrics['total_processed']:,}")
print(f"Final average: {processor.metrics['current_avg']:.2f}")
print(f"Peak value: {processor.metrics['peak_value']}")
print(f"Total anomalies: {processor.metrics['anomaly_count']}")

# Clean up
os.remove(temp_file)
print(f"\nCleaned up temporary file: {temp_file}")

## Key Takeaways

1. **ETL Pipeline Pattern**: Provides structured approach to data workflows with proper error handling and logging
2. **Vectorization**: Can provide 10-100x performance improvements for mathematical operations on large datasets
3. **Streaming Processing**: Essential for handling datasets larger than available memory
4. **Memory Management**: Use generators and iterators to process data efficiently without loading everything into memory

## Exercises

1. Create an ETL pipeline that processes multiple CSV files and combines them into a single output
2. Compare the performance of pandas `.apply()` vs vectorized operations on a large dataset
3. Implement a streaming word count algorithm that processes a large text file
4. Build a real-time anomaly detection system using sliding window statistics
5. Create a memory-efficient data aggregation system that processes data in configurable batch sizes